### Install PHATE

Potential of Heat-diffusion for Affinity-based Trajectory Embedding (PHATE) embeds high dimensional single-cell data into two or three dimensions for visualization of biological progressions.

In [ ]:
!pip install phate scprep

In [ ]:
import pandas as pd
import numpy as np
import phate
import os
import scprep
import matplotlib.pyplot as plt
import sklearn.preprocessing

data_path = "/Users/Mingyu/Desktop/Assignment4-part2/PHATE/datasets/"
print(data_path)

# matplotlib settings for Jupyter notebooks only
%matplotlib inline

### Load the dataset

In [ ]:
sparse=True
T1 = scprep.io.load_10X(os.path.join(data_path), sparse=sparse, gene_labels= "both")
pheno_data = pd.read_csv(os.path.join(data_path, "skeletal_pheno.tsv"),sep='\t')

### Preprocessing

Removing genes that are expressed in relatively few cells.

In [ ]:
T1 = scprep.filter.filter_rare_genes(T1, min_cells=10)

#### Normalization

In [ ]:
T1 = scprep.normalize.library_size_normalize(T1)

#### Transformation

In scRNA-seq analysis, the data is often log-transformed. This typically requires the addition of some small value to avoid taking log(0). We avoid this issue entirely by instead taking the square root transform. The square root function has a similar form as the log function with the added benefit of being stable at 0.

In [ ]:
T1 = scprep.transform.sqrt(T1)

### Embedding Data Using PHATE

We'll just use the default parameters for now, but the following parameters can be tuned (read our documentation at phate.readthedocs.io to learn more):

* `knn` : Number of nearest neighbors (default: 5). Increase this (e.g. to 20) if your PHATE embedding appears very disconnected. You should also consider increasing knn if your dataset is extremely large (e.g. >100k cells)
* `decay` : Alpha decay (default: 15). Decreasing decay increases connectivity on the graph, increasing decay decreases connectivity. This rarely needs to be tuned. Set it to None for a k-nearest neighbors kernel.
* `gamma` : Informational distance constant (default: 1). gamma=1 gives the PHATE log potential, but other informational distances can be interesting. If most of the points seem concentrated in one section of the plot, you can try gamma=0.

In [ ]:
phate_operator = phate.PHATE(n_jobs=-2)
Y_phate = phate_operator.fit_transform(T1)

#### Plotting PHATE with a scatter plot

In [ ]:
scprep.plot.scatter2d(Y_phate,c = ["%4.1f" % x for x in (pheno_data.loc[T1.index,'day'])] , figsize=(8,6), cmap="Spectral",
                      ticks=False, label_prefix="PHATE")

In [ ]:
#phate_operator.set_params(knn=10)
#Y_phate = phate_operator.fit_transform(T1)

In [ ]:
#scprep.plot.scatter2d(Y_phate,c = ["%4.1f" % x for x in (pheno_data.loc[T1.index,'day'])], 
#                      figsize=(8,6), cmap="Spectral",ticks=False, label_prefix="PHATE")

####  Visualize the embedding in 3D

In [ ]:
phate_operator.set_params(n_components=3)
Y_phate_3d = phate_operator.transform()
scprep.plot.scatter3d(Y_phate_3d, c = ["%4.1f" % x for x in (pheno_data.loc[T1.index,'day'])]  , figsize=(8,6), cmap="Spectral",
                      ticks=False, label_prefix="PHATE")


#### Create a gif showing the rotating 3D plot

In [ ]:
scprep.plot.rotate_scatter3d(Y_phate_3d, c = ["%4.1f" % x for x in (pheno_data.loc[T1.index,'day'])] , 
                             figsize=(8,6), cmap="Spectral",
                             ticks=False, label_prefix="PHATE")

# to save as a gif:
# >>> scprep.plot.rotate_scatter3d(Y_phate_3d, c=sample_labels, 
# ...                              figsize=(8,6), cmap="Spectral",
# ...ticks=False, label_prefix="PHATE", filename="phate.gif")
# to save as an mp4:
# >>> scprep.plot.rotate_scatter3d(Y_phate_3d, c=sample_labels, 
# ...                              figsize=(8,6), cmap="Spectral",
# ...                              ticks=False, label_prefix="PHATE", filename="phate.mp4")

### Comparison with other visualization tools

In [ ]:
import sklearn.decomposition # PCA
import sklearn.manifold # t-SNE
import umap #UMAP
import time


start = time.time()
pca_operator = sklearn.decomposition.PCA(n_components=2)
Y_pca = pca_operator.fit_transform(np.array(T1))
end = time.time()
print("Embedded PCA in {:.2f} seconds.".format(end-start))

start = time.time()
pca_operator = sklearn.decomposition.PCA(n_components=100)
tsne_operator = sklearn.manifold.TSNE(n_components=2)
Y_tsne = tsne_operator.fit_transform(pca_operator.fit_transform(np.array(T1)))
end = time.time()
print("Embedded t-SNE in {:.2f} seconds.".format(end-start))

start = time.time()
Y_umap = umap.UMAP().fit_transform(pca_operator.fit_transform(np.array(T1)))
end = time.time()
print("Embedded UMAP in {:.2f} seconds.".format(end-start))

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(16, 5))

#plotting PCA
scprep.plot.scatter2d(Y_pca, label_prefix="PC", title="PCA",
                      c = ["%4.1f" % x for x in (pheno_data.loc[T1.index,'day'])] , ticks=False, cmap='Spectral', ax=ax1)

#plotting tSNE
scprep.plot.scatter2d(Y_tsne, label_prefix="t-SNE", title="t-SNE", legend=False,
                      c = ["%4.1f" % x for x in (pheno_data.loc[T1.index,'day'])] , ticks=False, cmap='Spectral', ax=ax2)

#plotting UMAP
scprep.plot.scatter2d(Y_umap, label_prefix="UMAP", title="UMAP", legend=False,
                      c = ["%4.1f" % x for x in (pheno_data.loc[T1.index,'day'])] , ticks=False, cmap='Spectral', ax=ax3)

#plotting PHATE
scprep.plot.scatter2d(Y_phate, label_prefix="PHATE", title="PHATE", legend=False,
                      c = ["%4.1f" % x for x in (pheno_data.loc[T1.index,'day'])] , ticks=False, cmap='Spectral', ax=ax4)


plt.tight_layout()
plt.show()

## Available in scanpy (as a external API)

In [ ]:
import scanpy as sc
import scanpy.external as sce

In [ ]:
MOCAdata = sc.read_h5ad(os.path.join(data_path, "MOCA_results.h5ad"))

In [ ]:
sce.tl.phate(MOCAdata)

In [ ]:
plt.rcParams['figure.figsize'] = (5,5)

sce.pl.phate(MOCAdata, color='louvain')

In [ ]:
sc.pl.tsne(MOCAdata, color='louvain')

In [ ]:
sc.pl.umap(MOCAdata, color='louvain')

In [ ]:
MOCAdata.uns['iroot'] = np.argmax(MOCAdata.obsm['X_phate'][:,0])

In [ ]:
sc.tl.diffmap(MOCAdata)

In [ ]:
sc.tl.dpt(MOCAdata, n_branchings=2)

In [ ]:
sce.pl.phate(MOCAdata, color = 'dpt_pseudotime')

In [ ]:
sce.pl.phate(MOCAdata, color = 'dpt_groups')

In [ ]:
ax = scprep.plot.jitter(MOCAdata.obs['leiden'], 
                        MOCAdata.obs['dpt_pseudotime'], 
                        c=MOCAdata.obs['dpt_groups'], 
                        legend_anchor=(1,1),
                       figsize=(8,4))

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
scprep.plot.utils.shift_ticklabels(ax.xaxis, dx=0.15)